Requête 1: liste des acteurs par film


In [4]:
# import des librairies
import os
import pandas as pd 
import numpy as np

In [12]:

# lire les fichiers .tsv vers un dataframe
df_name_basics = pd.read_csv('../name.basics.tsv', sep="\t", nrows = 80000, usecols=["nconst", "primaryName"])

#df_name_basics = df_name_basics[["nconst", "primaryName", "knownForTitles"]]
#
df_principals = pd.read_csv('../title.principals.tsv', sep="\t", nrows = 10000) 
#
df_principals = df_principals.loc[(df_principals['category']=="actor")|(df_principals['category']=="actress"),:]
df_principals = df_principals[["tconst", "nconst"]]
# inner join with merge
df_merged_inner = pd.merge(left = df_principals, right= df_name_basics, left_on='nconst', right_on='nconst')
#lire title.basics


In [43]:
# lire le fichier title.basics.tsv
df_title_basics = pd.read_csv('../title.basics.tsv', sep="\t", nrows = 80000)
# mettre un filtre sur titleType
df_title_basics = df_title_basics.loc[(df_title_basics['titleType']=="tvMovie")|(df_title_basics['titleType']=="movie"),:]
# select interesting columns
df_title_filtred = df_title_basics[["tconst", "primaryTitle"]] 
#faire un merge
liste_actor = pd.merge(left=df_title_filtred, right = df_merged_inner, left_on = 'tconst', right_on = 'tconst', how = 'inner')
liste_actor = liste_actor[["primaryTitle","primaryName"]]
#
#liste_actor.to_csv('liste_actor.csv', sep =",")
#groupby
liste_actors = liste_actor.groupby(['primaryTitle'])['primaryName'].apply(','.join).reset_index()
liste_actors.to_csv('liste_actors.csv', sep =",")
#print(liste_actors.head())

In [18]:
df_title_basics = pd.read_csv('../title.basics.tsv', sep="\t", nrows = 1000)
#df_title_film = df_title_basics[["tconst", "primaryTitle"]]
#print(df_title_film.head())

In [16]:
df_merge_film = df_title_film.merge(df_merge_filt)
print(df_merge_film.head())

tconst                                 primaryTitle       primaryName
0  tt0000005                             Blacksmith Scene    Charles Kayser
1  tt0000005                             Blacksmith Scene          John Ott
2  tt0000007  Corbett and Courtney Before the Kinetograph  James J. Corbett
3  tt0000007  Corbett and Courtney Before the Kinetograph    Peter Courtney
4  tt0000008       Edison Kinetoscopic Record of a Sneeze          Fred Ott


In [10]:
df_title_akas = pd.read_csv('../title.akas.tsv', sep="\t", nrows =1000)
print(df_title_akas.head())

titleId  ordering                      title region language  \
0  tt0000001         1  Carmencita - spanyol tánc     HU       \N   
1  tt0000001         2                 Καρμενσίτα     GR       \N   
2  tt0000001         3                 Карменсита     RU       \N   
3  tt0000001         4                 Carmencita     US       \N   
4  tt0000001         5                 Carmencita     \N       \N   

         types attributes  isOriginalTitle  
0  imdbDisplay         \N                0  
1           \N         \N                0  
2           \N         \N                0  
3           \N         \N                0  
4     original         \N                1  


In [12]:
print(df1[["title"]].head())

title
0  Carmencita - spanyol tánc
1                 Καρμενσίτα
2                 Карменсита
3                 Carmencita
4                 Carmencita


In [21]:
df2 = pd.read_csv('../title.basics.tsv', sep="\t", nrows = 1000)
print(df2.head())

tconst titleType            primaryTitle           originalTitle  \
0  tt0000001     short              Carmencita              Carmencita   
1  tt0000002     short  Le clown et ses chiens  Le clown et ses chiens   
2  tt0000003     short          Pauvre Pierrot          Pauvre Pierrot   
3  tt0000004     short             Un bon bock             Un bon bock   
4  tt0000005     short        Blacksmith Scene        Blacksmith Scene   

   isAdult  startYear endYear runtimeMinutes                    genres  
0        0       1894      \N              1         Documentary,Short  
1        0       1892      \N              5           Animation,Short  
2        0       1892      \N              4  Animation,Comedy,Romance  
3        0       1892      \N             \N           Animation,Short  
4        0       1893      \N              1              Comedy,Short  


In [5]:
df3 = pd.read_csv('../title.principals.tsv', sep="\t", nrows =1000)
print(df3.head())


tconst  ordering     nconst         category                      job  \
0  tt0000001         1  nm1588970             self                       \N   
1  tt0000001         2  nm0005690         director                       \N   
2  tt0000001         3  nm0374658  cinematographer  director of photography   
3  tt0000002         1  nm0721526         director                       \N   
4  tt0000002         2  nm1335271         composer                       \N   

    characters  
0  ["Herself"]  
1           \N  
2           \N  
3           \N  
4           \N  


In [33]:
df_actor = df3.loc[df3['category']=="actor",:]
#print(df_actor.head())

In [37]:
df4 = pd.read_csv('../title.ratings.tsv', sep="\t", nrows = 1000)
print(df4.head())

tconst  averageRating  numVotes
0  tt0000001            5.6      1550
1  tt0000002            6.1       186
2  tt0000003            6.5      1207
3  tt0000004            6.2       113
4  tt0000005            6.1      1934


In [13]:
df1 = pd.read_csv('../title.basics.tsv', sep="\t", nrows = 500)
print(df1.head())

      tconst titleType            primaryTitle           originalTitle  \
0  tt0000001     short              Carmencita              Carmencita   
1  tt0000002     short  Le clown et ses chiens  Le clown et ses chiens   
2  tt0000003     short          Pauvre Pierrot          Pauvre Pierrot   
3  tt0000004     short             Un bon bock             Un bon bock   
4  tt0000005     short        Blacksmith Scene        Blacksmith Scene   

   isAdult  startYear endYear runtimeMinutes                    genres  
0        0       1894      \N              1         Documentary,Short  
1        0       1892      \N              5           Animation,Short  
2        0       1892      \N              4  Animation,Comedy,Romance  
3        0       1892      \N             \N           Animation,Short  
4        0       1893      \N              1              Comedy,Short  


In [18]:
del(df1)